<a href="https://colab.research.google.com/github/MorningStarTM/brain_cancer_detection/blob/main/brain_tumor_classification_with_simple_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [344]:
#importing libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle
from glob import glob
from tensorflow.keras.layers import*
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import*
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [345]:
#parameters
H, W = 224, 224
channel = 3
num_class = 3
batch_size = 16
class_names = ["glioma", "meningioma", "notumor", "pituitary"]

#Data pipeline

In [346]:
def load_data(path, split=0.1):
    images = shuffle(glob(os.path.join(path, "*", "*.jpg")))
    
    split_size = int(len(images) * split)
    #split the data
    train_data, valid_data = train_test_split(images, test_size=split_size, random_state=42)
    train_data, test_data = train_test_split(train_data, test_size=split_size, random_state=42)

    return train_data, valid_data, test_data 

In [372]:
def process_image(path):
    #decode the path
    path = path.decode()
    #read image
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    #resize the image
    image = cv2.resize(image, [224, 224])
    #scale the image
    image = image / 255.0
    image = image.astype(np.float32)

    #labeling the image
    class_name = path.split("/")[-2]
    class_idx = class_names.index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return image, class_idx

In [373]:
def parse(path):
    image, labels = tf.numpy_function(process_image, [path], (tf.float32, tf.int32))
    labels = tf.one_hot(labels, 4)
    image.set_shape([224, 224, 3])
    labels.set_shape(4)
  
    return image, labels

In [374]:
#tensorflow dataset
def tf_dataset(images, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((images))
    dataset = dataset.map(parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(8)
    return dataset

#Model Creation

In [375]:
path = "/content/drive/MyDrive/DataSet/brain_cancer/Training"

In [376]:
x_train, x_valid, x_test = load_data(path)
print(f"Train:{len(x_train)} - Test:{len(x_test)} - Valid:{len(x_valid)}")

Train:4586 - Test:573 - Valid:573


In [377]:
df = tf_dataset(x_train)